# Change to the root directory

In [20]:
import os

In [21]:
directory_path = 'C:/WINE_QUALITY_ML_PROJ'

In [22]:
os.chdir(directory_path)

In [23]:
%pwd

'C:\\WINE_QUALITY_ML_PROJ'

# Create the entity (class skeleton)

In [24]:
from dataclasses import dataclass
from pathlib import Path

In [25]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: str
    unzip_data_dir: Path
    all_schema: dict # reading from schema.yaml, where we already store that what were the cols(features) present at the time of training

# create source configuration

similar to data ingestion

In [26]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [31]:
# the only thing to update in configuration is def of te func get_data_validation_config
# because ther eis we already made this constructor
# here we define the same constructor again so we can run entire code here to check whether it is working fine or not

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH,
            schema_file_path = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath) # get the whole configuration
        self.params = read_yaml(params_file_path) # get the schema
        self.schema = read_yaml(schema_file_path) # get the parameters

        create_directories([self.config.artifacts_root]) # create artifacts folder, if exist already then don't throw error, handle this in utils.common.create_directories code


    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation # get the configuration for data validation from whole configuration
        schema = self.schema.COLUMNS # get the schema used in training

        create_directories([config.root_dir]) # create the root_dir i.e. 'data validation' folder inside artifacts folder bcoz it' holding path -> artifacts/data_validation

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            status_file = config.status_file,
            unzip_data_dir = config.unzip_data_dir,
            all_schema = schema
        )

        return data_validation_config # return the object

# create the component

In [28]:
import pandas as pd

In [29]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            overall_validation_status = True

            data = pd.read_csv(self.config.unzip_data_dir) # data we are fetching now
            data_cols = data.dtypes.to_dict() # data type of cols along with cols

            all_schema = self.config.all_schema # data used at the time of training

            for column_name, data_type in data_cols.items():

                if ((column_name in all_schema) and (all_schema[column_name] == data_type)): # if col is present and it's type is also matching
                    val_status = True
                    
                    with open(self.config.status_file, 'a') as f:
                        f.write(f"Col Name: {column_name}, Col type: {data_type} \nValidation Status: {val_status} \n")

                elif ((column_name in all_schema) and (all_schema[column_name] != data_type)): # col is present but type is not matching
                    val_status = False
                    overall_validation_status = False
                    
                    with open(self.config.status_file, 'a') as f:
                        f.write(f"Col Name: {column_name}, Col type: {data_type}, Existing_Col_type: {all_schema[column_name]} \nValidation Status: {val_status} \n")
                    raise Exception('Data type is not matching, terminating the further data fetching process')

                else: # col is not present
                    val_status = False
                    overall_validation_status = False
                    
                    with open(self.config.status_file, 'a') as f:
                        f.write(f"Col Name: {column_name}, Col type: {data_type} \nValidation Status: {val_status} \n")
                    raise Exception("Column does not exist, terminating the further data fetching process")

            return overall_validation_status
        
        except Exception as e:
            print(e)

# create pipeline

In [30]:
try:
    config = ConfigurationManager() # creating an object so calling the configuration manager
    # here don't be confused like in the ConfigurationManager Class Constructor we defined 3 arguments but here we are not passing any, So why it is not showing error?
    # Because we assign all 3 arguments with "default" values i.e. path of config, params and schema yaml files
    data_validation_config = config.get_data_validation_config() # fetching the object from config manager
    data_validation = DataValidation(config = data_validation_config) # creating the component related to object, only make 1 component
    data_validation.validate_all_columns() # fetching that component
except Exception as e:
    raise e

[2024-03-02 14:38:17,056: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-02 14:38:17,056: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-02 14:38:17,060: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-02 14:38:17,068: INFO: common: created directory at: artifacts]
[2024-03-02 14:38:17,070: INFO: common: created directory at: artifacts/data_validation]
